### Laden benötigter Zusatzbibliotheken

In [3]:
import pandas as pd

### Importieren und weiteres Prozessieren der eben abgefragten Daten


In [6]:
df = pd.read_csv("Studentenbewegung.csv", encoding="utf-8")
df

,idn,titel,author,rela,gnd
0,1337043966,Ausdrucksstarker Sprachgebrauch als Katalysato...,unknown,unknown,unknown
1,1325298123,Burg in der Brandung?,"Gilcher-Holtey, Ingrid",Verfasser,unknown
2,1338119109,Der Vietnamkrieg und die 68er Bewegung. Ursach...,"Schröders, Katharina",Verfasser,unknown
3,1337965448,Der Vietnamkrieg und die 68er Bewegung. Ursach...,"Schröders, Katharina",Verfasser,unknown
4,1334711879,Proteste homosexueller Studierender in Münster,"Schäfers, Norbert",Verfasser,unknown
...,...,...,...,...,...
987,1333866011,Rudi Dutschke: Tragische Folge eines Attentats,"Meißner, Thomas",Verfasser,unknown
988,362816468,Stimmen aus der deutschen christlichen Student...,unknown,unknown,unknown
989,362816565,Stimmen und Zeugnisse,unknown,unknown,unknown
990,122532694X,Türkische und Türkisch-Deutsche Perspektiven i...,"Gezen, Ela",Verfasser,unknown


Das Feld "titel" enthält den Titel der Publikation, das Feld "rela" gibt an, in welcher Relation die genannte Person zur Publikation steht (bspw. Verfasser, Herausgeber, Übersetzer etc.). Da beide Felder für die weitere Analyse der Autoren nicht relevant sind, werden sie aus der Tabelle entfernt: 

In [8]:
df = df.drop(['titel', 'rela'], axis=1)
df

,idn,author,gnd
0,1337043966,unknown,unknown
1,1325298123,"Gilcher-Holtey, Ingrid",unknown
2,1338119109,"Schröders, Katharina",unknown
3,1337965448,"Schröders, Katharina",unknown
4,1334711879,"Schäfers, Norbert",unknown
...,...,...,...
987,1333866011,"Meißner, Thomas",unknown
988,362816468,unknown,unknown
989,362816565,unknown,unknown
990,122532694X,"Gezen, Ela",unknown


Als nächstes werden die Suchtreffer auf Ergebnisse reduziert, bei denen das Feld author befüllt ist. Wie vorhin betrachtet, werden bspw. Herausgeber oder weitere Autoren in einem anderen Marc-Feld hinterlegt. 

In [10]:
df = df[df['author'] != 'unknown']
df

,idn,author,gnd
1,1325298123,"Gilcher-Holtey, Ingrid",unknown
2,1338119109,"Schröders, Katharina",unknown
3,1337965448,"Schröders, Katharina",unknown
4,1334711879,"Schäfers, Norbert",unknown
5,1334711895,"Jüttemann, Veronika",(DE-588)13015119X; https://d-nb.info/gnd/13015...
...,...,...,...
981,1179517342,"Schmidt, Rudi",unknown
982,1161721533,"Wolfrum, Edgar",(DE-588)122851595; https://d-nb.info/gnd/12285...
984,1233275909,"Bührer, Werner",(DE-588)170414434; https://d-nb.info/gnd/17041...
987,1333866011,"Meißner, Thomas",unknown


### Bereinigen: 

Im folgenden Schritt wird das GND-Feld modifiziert. In diesem Feld ist die GND-Nummer für den jeweiligen Autor, sofern vorhanden, hinterlegt, allerdings auf unterschiedliche Weise:
- (DE-588)XXXXXXX
- die Purl des Eintrags nach dem Muster https://d-nb.info/gnd/XXXXXX
- (DE-101)XXXXXXX

Die einzelnen Werte sind per Semikolon getrennt, die untenstehende Funktion trennt alles hinter dem ersten Semikolon ab und behält nur den ersten Wert - im dafür neu geschaffenen Feld gnd_modified.

In [12]:
df.loc[:, 'gnd_modified'] = df['gnd'].str.split(';').str.get(0)
df

C:\Users\Palek\AppData\Local\Temp\ipykernel_956\2604930092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'gnd_modified'] = df['gnd'].str.split(';').str.get(0)


,idn,author,gnd,gnd_modified
1,1325298123,"Gilcher-Holtey, Ingrid",unknown,unknown
2,1338119109,"Schröders, Katharina",unknown,unknown
3,1337965448,"Schröders, Katharina",unknown,unknown
4,1334711879,"Schäfers, Norbert",unknown,unknown
5,1334711895,"Jüttemann, Veronika",(DE-588)13015119X; https://d-nb.info/gnd/13015...,(DE-588)13015119X
...,...,...,...,...
981,1179517342,"Schmidt, Rudi",unknown,unknown
982,1161721533,"Wolfrum, Edgar",(DE-588)122851595; https://d-nb.info/gnd/12285...,(DE-588)122851595
984,1233275909,"Bührer, Werner",(DE-588)170414434; https://d-nb.info/gnd/17041...,(DE-588)170414434
987,1333866011,"Meißner, Thomas",unknown,unknown


Nachdem gnd_modified als Feld hinzugefügt wurde, wird im nächsten Schritt das Feld gnd entfernt.

In [14]:
df = df.drop(['gnd'], axis=1)
df

,idn,author,gnd_modified
1,1325298123,"Gilcher-Holtey, Ingrid",unknown
2,1338119109,"Schröders, Katharina",unknown
3,1337965448,"Schröders, Katharina",unknown
4,1334711879,"Schäfers, Norbert",unknown
5,1334711895,"Jüttemann, Veronika",(DE-588)13015119X
...,...,...,...
981,1179517342,"Schmidt, Rudi",unknown
982,1161721533,"Wolfrum, Edgar",(DE-588)122851595
984,1233275909,"Bührer, Werner",(DE-588)170414434
987,1333866011,"Meißner, Thomas",unknown


In wie vielen Feldern ist die GND-Nummer nicht hinterlegt?

In [16]:
count_unknown = (df['gnd_modified'] == 'unknown').sum()
print("Einträge in gnd_modified mit value 'unknown': ", count_unknown)

Einträge in gnd_modified mit value 'unknown':  234


Das Dokument wird nun unter dem Titel authors.csv und mit UTF-8-Codierung exportiert.

In [21]:
df.to_csv("authors.csv", encoding="utf-8-sig", index=False)